In [1]:
import os as os
import sys as sys
import re as re
import pandas as pd
import numpy as np
import json as json
import pickle
import urllib as urllib
import zlib as zlib
import base64 as base64
from requests import Request, Session
import requests
from numpy import trapz
import itertools
import zlib as zlib
import base64 as base64
import pickle
import pathlib
from tqdm import tqdm
from datetime import datetime
import time
import copy
from Bio import SeqIO
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.dummy import DummyClassifier

pd.options.display.max_columns = 100
pd.options.display.min_rows = None
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 100

DATA_PATH = pathlib.Path('/data1/home/adpatter/gene-to-phenotype-predictions/adpatter/data/')
PROTEIN_PATH = DATA_PATH.joinpath('gene_symbol_protein_sequences.pkl')
EFFECT_SIZE_PATH = DATA_PATH.joinpath('capstone_body_weight_Statistical_effect_size_analysis_genotype_early_adult_scaled_13022023_gene_symbol_harmonized.pkl')

In [2]:
df_protein = pd.read_pickle(PROTEIN_PATH)
df_effect_size = pd.read_pickle(EFFECT_SIZE_PATH)

In [3]:
df = df_effect_size.copy()

df = df.groupby(['gene_symbol_harmonized'], group_keys=False).apply(lambda x: x.loc[x['est_m_ea'] == x['est_m_ea'].max()])

assert not df['gene_symbol_harmonized'].duplicated().any()

df_effect_size = df.copy()

In [4]:
df = df_protein.copy()

df = df[['gene_symbol_harmonized', 'seq']].drop_duplicates()

df = df.groupby(['gene_symbol_harmonized'], group_keys=False).apply(lambda x: x.loc[x['seq'].str.len() == x['seq'].str.len().max()].sample(n=1))

assert not df['gene_symbol_harmonized'].duplicated().any()

df_protein_max_seq = df.copy()

In [6]:
df = df_protein_max_seq.copy()

df = df.merge(df_effect_size, on='gene_symbol_harmonized', how='inner')

df = df[['gene_symbol_harmonized', 'est_m_ea', 'seq']]

assert not df['gene_symbol_harmonized'].duplicated().any()

assert not df.isna().any().any()

df.to_pickle(DATA_PATH.joinpath('protein_sequences_effect_size_sample.pkl'))